In [74]:
import pandas as pd 
import os
from difflib import SequenceMatcher
import ace_tools as tools

In [3]:
print(os.getcwd())

/Users/arhanjoshi/Desktop/Data Analyst Work/CSV_data_wrangling/Scopus_data/scripts


In [75]:
file_path = r"../data/processed/Unique Affiliation.xlsx"
df = pd.read_excel(file_path)

In [76]:
df.head()

,Affiliation,Count,Percentage
0,"National Renewable Energy Laboratory, Golden, ...",1936,0.001968
1,"Naval Research Laboratory, Washington, DC 2037...",1701,0.001729
2,"AT and T Bell Laboratories, Murray Hill, NJ 07...",1550,0.001576
3,"Sandia National Laboratories, Albuquerque, NM ...",1453,0.001477
4,"Bell Laboratories, Murray Hill, NJ 07974, Unit...",1311,0.001333


In [ ]:
df['Affiliation'].str.split(',')

In [ ]:
List(df['Affiliation'].iloc[281502: 281504])                     

In [56]:
list(df['Affiliation'].iloc[281502: 281504])

['Zyvex Instruments, Richardson, TX, 75081, United States, Siemens Energy and Automation, Arlington, TX, United States',
 'Zyvex Instruments, Richardson, TX, 75081, United States']

In [21]:
list(df["Affiliation"][:5])

['National Renewable Energy Laboratory, Golden, 80401, CO, United States',
 'Naval Research Laboratory, Washington, DC 20375, United States',
 'AT and T Bell Laboratories, Murray Hill, NJ 07974, United States',
 'Sandia National Laboratories, Albuquerque, NM 87185, United States',
 'Bell Laboratories, Murray Hill, NJ 07974, United States']

In [16]:
df.shape

(281505, 3)

In [14]:
df['Affiliation'] = df["Affiliation"].fillna("")

Index(['Affiliation', 'Count', 'Percentage'], dtype='object')

In [15]:
fields = [
    {"field": "Affiliation", "type": "String"}
]

In [57]:
df.head()


,Affiliation,Count,Percentage
0,"National Renewable Energy Laboratory, Golden, ...",1936,0.001968
1,"Naval Research Laboratory, Washington, DC 2037...",1701,0.001729
2,"AT and T Bell Laboratories, Murray Hill, NJ 07...",1550,0.001576
3,"Sandia National Laboratories, Albuquerque, NM ...",1453,0.001477
4,"Bell Laboratories, Murray Hill, NJ 07974, Unit...",1311,0.001333


In [ ]:
affliation = list(df['Affiliation'].str.split(',')[0])
affliation

['National Renewable Energy Laboratory',
 ' Golden',
 ' 80401',
 ' CO',
 ' United States']

first_parts = df['Affiliation'].apply(lambda s: s.split(',')[0])


In [60]:
df.head()

,Affiliation,Count,Percentage
0,"National Renewable Energy Laboratory, Golden, ...",1936,0.001968
1,"Naval Research Laboratory, Washington, DC 2037...",1701,0.001729
2,"AT and T Bell Laboratories, Murray Hill, NJ 07...",1550,0.001576
3,"Sandia National Laboratories, Albuquerque, NM ...",1453,0.001477
4,"Bell Laboratories, Murray Hill, NJ 07974, Unit...",1311,0.001333


In [70]:
first_parts = df['Affiliation'].apply(lambda s: s.split(',')[0].strip().lower().replace(" ","").replace('&','and'))

In [73]:
first_parts.nunique()

66659

In [82]:
# Sequence Master

# convert excel file to csv file and save
df_aff_excel = pd.read_excel('../data/processed/Unique Affiliation.xlsx')
output_path = '../data/processed/unique_affiliation_uncleaned.csv'
to_csv = df_aff.to_csv(output_path, index=False)

In [86]:
# load csv
df_aff = pd.read_csv('../data/processed/unique_affiliation_uncleaned.csv')
df_aff.head()

,Affiliation,Count,Percentage
0,"National Renewable Energy Laboratory, Golden, ...",1936,0.00196822776136458
1,"Naval Research Laboratory, Washington, DC 2037...",1701,0.00172931581719068
2,"AT and T Bell Laboratories, Murray Hill, NJ 07...",1550,0.00157580218497681
3,"Sandia National Laboratories, Albuquerque, NM ...",1453,0.00147718746759439
4,"Bell Laboratories, Murray Hill, NJ 07974, Unit...",1311,0.0013328236545191


In [87]:
# prepare for clustering deduplication

affliation_list = df['Affiliation'].tolist()
threshold = 0.90
assigned = set()
clusters = []

In [ ]:
for aff in affliation_list:
    if aff in assigned:
        continue
    #start new cluster
    cluster = [aff]
    assigned.add(aff)
    for other in affliation_list:
        if other in assigned:
            continue
        if SequenceMatcher(None, aff, other).ratio() >= threshold:
            cluster.append(other)
            assigned.add(other)
    clusters.append(cluster)

In [ ]:
# Build mapping of each original to its canonical form (shortest string)

records = []
for idx, cluster in enumerate(clusters):
    canonical = max(clusters, key=len)
    for memeber in cluster:
        records.append({
            'Original Affiliation': member,
            'Canonical Affiliation': canonical,
            'Cluster ID': idx
        })

mapping_df = pd.DataFrame(records)

In [ ]:
# save to excel
output_path_excel = '../data/processed/unique_affiliation_dedupe.xslx'
mapping_df.to_excel(output_path_excel, index = False)

In [ ]:
# display mapping to user
tools.display_dataframe_to_user("Deduplicated Affiliation Mapping", mapping_df)

print(f"Excel file created: {output_path_excel}")